In [52]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity


import warnings; warnings.simplefilter('ignore')

In [53]:
import pandas as pd

data= pd.read_excel(r"C:\Users\ccraq\OneDrive\Documents\PYTHON\projet_python\dataset.xlsx")
data

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124.0,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-09-06 00:00:00,5562.0,6.5,2015.0,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120.0,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185.0,7.1,2015.0,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119.0,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480.0,6.3,2015.0,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136.0,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292.0,7.5,2015.0,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137.0,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,2015-01-04 00:00:00,2947.0,7.3,2015.0,1.747999e+08,1.385749e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10873,21,tt0060371,0.080598,0,0,The Endless Summer,Michael Hynson|Robert August|Lord 'Tally Ho' B...,NaN,Bruce Brown,NaN,...,"The Endless Summer, by Bruce Brown, is one of ...",95.0,Documentary,Bruce Brown Films,6/15/1966,11.0,7.4,1966.0,0.000000e+00,0.000000e+00
10874,20379,tt0060472,0.065543,0,0,Grand Prix,James Garner|Eva Marie Saint|Yves Montand|Tosh...,NaN,John Frankenheimer,Cinerama sweeps YOU into a drama of speed and ...,...,Grand Prix driver Pete Aron is fired by his te...,176.0,Action|Adventure|Drama,Cherokee Productions|Joel Productions|Douglas ...,12/21/1966,20.0,5.7,1966.0,0.000000e+00,0.000000e+00
10875,39768,tt0060161,0.065141,0,0,Beregis Avtomobilya,Innokentiy Smoktunovskiy|Oleg Efremov|Georgi Z...,NaN,Eldar Ryazanov,NaN,...,An insurance agent who moonlights as a carthie...,94.0,Mystery|Comedy,Mosfilm,1966-01-01 00:00:00,11.0,6.5,1966.0,0.000000e+00,0.000000e+00
10876,21449,tt0061177,0.064317,0,0,"What's Up, Tiger Lily?",Tatsuya Mihashi|Akiko Wakabayashi|Mie Hama|Joh...,NaN,Woody Allen,WOODY ALLEN STRIKES BACK!,...,"In comic Woody Allen's film debut, he took the...",80.0,Action|Comedy,Benedict Pictures Corp.,1966-02-11 00:00:00,22.0,5.4,1966.0,0.000000e+00,0.000000e+00


In [63]:
data_cbr=data.copy()
data_cbr['tagline']=data_cbr['tagline'].astype('str')
data_cbr['overview']=data_cbr['overview'].astype('str')

data_cbr['tagline'] = data_cbr['tagline'].fillna('')
data_cbr['description'] = data_cbr['overview'] + data_cbr['tagline']
data_cbr['description'] = data_cbr['description'].fillna('')

In [64]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(data_cbr['description'])
data_cbr.shape

(10878, 22)

In [65]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [66]:
cosine_sim[0]

array([1.        , 0.00218801, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [67]:
#function that returns the 30 most similar movies based on the cosine similarity score.

data_cbr = data_cbr.reset_index()
titles = data_cbr['original_title']
indices = pd.Series(data_cbr.index, index=data_cbr['original_title'])

In [68]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [70]:
#ce que l'on va lier à streamlit

get_recommendations('The Godfather').head(10)

9770     The Godfather: Part II
5484                 The Family
5594                 Blood Ties
2858                       Made
7960         Johnny Dangerously
2571             American Movie
9430                   Mobsters
1010             Bad Turn Worse
9996    The Godfather: Part III
2542              Summer of Sam
Name: original_title, dtype: object

In [22]:
get_recommendations('The Dark Knight').head(10)

4369                                The Dark Knight Rises
8091                                       Batman Forever
8254                                       Batman Returns
9193                                               Batman
3572                                     Batman: Year One
2026                           Batman: Under the Red Hood
5470              Batman: The Dark Knight Returns, Part 2
3248    Batman Unmasked: The Psychology of the Dark Kn...
6337                                The Batman vs Dracula
8738                   Batman Beyond: Return of the Joker
Name: original_title, dtype: object